In [1]:
import torch
from torch import nn,optim
from torch.utils.data import Dataset,TensorDataset,DataLoader
import tqdm

import numpy as np

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
import time

### imageの準備

In [2]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar xf 102flowers.tgz
!mkdir oxford-102
!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg

--2021-06-19 20:50:52--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
www.robots.ox.ac.uk (www.robots.ox.ac.uk) をDNSに問いあわせています... 129.67.94.2
www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 301 Moved Permanently
場所: https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz [続く]
--2021-06-19 20:50:53--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 344862509 (329M) [application/x-gzip]
`102flowers.tgz' に保存中

102flowers.tgz      100%[===================>] 328.89M  3.91MB/s    in 1m 59s  

2021-06-19 20:52:52 (2.77 MB/s) - `102flowers.tgz' へ保存完了 [344862509/344862509]



In [3]:
img_data = ImageFolder("./oxford-102/",
                     transform=transforms.Compose([transforms.Resize(80),transforms.CenterCrop(64),transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data,batch_size = batch_size, shuffle = True)

### GnetとDnetの定義

In [4]:
nz = 100
ngf = 32

class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*2,ngf*1,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*1,3,4,2,1,bias=True),
            nn.Tanh()
            )

    def forward(self,x):
        out = self.main(x)
        return out

In [5]:
ndf=32

class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3,ndf,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf,ndf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*4,ndf*8,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*8,1,4,1,0,bias=True)
            )

    def forward(self,x):
        out = self.main(x)
        return out.squeeze()

In [6]:
d = DNet().to("cuda:0")
g = GNet().to("cuda:0")

opt_d = optim.Adam(d.parameters(),lr=0.0002,betas=(0.5,0.999))
opt_g = optim.Adam(g.parameters(),lr=0.0002,betas=(0.5,0.999))

ones = torch.ones(batch_size).to("cuda:0")
zeros = torch.zeros(batch_size).to("cuda:0")

loss_f = nn.BCEWithLogitsLoss()

fixed_z = torch.randn(batch_size,nz,1,1).to("cuda:0")
gif_z = torch.randn(1,nz,1,1).to("cuda:0")

### 訓練関数の定義

In [7]:
from statistics import mean

def train_dcgan(g,d,opt_g,opt_d,loader):
    log_loss_g = []
    log_loss_d = []
    
    for real_img , _ in tqdm.tqdm(loader):
        batch_len = len(real_img)
        real_img = real_img.to("cuda:0")
        z = torch.randn(batch_len,nz,1,1).to("cuda:0")
        #------------------------------------------------------------------------------------------------------------------
        fake_img = g(z)                                                         #フェイクイメージの作成
        fake_img_tensor = fake_img.detach()                  #フェイクイメージを計算グラフから分離
        out = d.forward(fake_img)                                      #フェイクイメージの判定
        loss_g = loss_f(out, ones[:batch_len])                 #フェイク画像の損失計算 as 本物
        log_loss_g.append(loss_g.item())                        #
        d.zero_grad() , g.zero_grad()                               #勾配初期化
        loss_g.backward()                                                  #生成機の逆伝搬
        opt_g.step()                                                           #生成機の更新
        #------------------------------------------------------------------------------------------------------------------
        real_out = d.forward(real_img)                                       #本物画像の判定
        loss_d_real = loss_f(real_out,ones[:batch_len])         #本物画像の損失計算

        fake_img = fake_img_tensor                                          #
        fake_out = d.forward(fake_img_tensor)                      #
        loss_d_fake = loss_f(fake_out, zeros[:batch_len])    #フェイク画像の損失計算 as フェイク

        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())
        d.zero_grad(),g.zero_grad()                                         #勾配初期化
        loss_d.backward()                                                          #判別機の逆伝搬
        opt_d.step()                                                                   #判別機の更新
        #------------------------------------------------------------------------------------------------------------------
    return mean(log_loss_g),mean(log_loss_d)

### 訓練開始

In [8]:
import os
if not os.path.exists("data"):
    os.mkdir("data")

In [9]:
from PIL import Image
import glob

total_time=0.0
average_time=0.0
iter_epoch=100

gif_data = []

for epoch in range(iter_epoch):
    t_start=time.time()
    print("current epoch : ",epoch)
    train_dcgan(g,d,opt_g,opt_d,img_loader)
    t_finish=time.time()
    total_time+=t_finish-t_start
    average_time=total_time/(epoch+1)
    print("runtime = ",t_finish-t_start,"sec")
    print("expected remaining time = ",int(average_time*(iter_epoch-(epoch+1))),"sec",":",int(average_time*(iter_epoch-(epoch+1))/60),"minute")
    gif_data.append(g(gif_z))
    if epoch%1==0:
        torch.save(g.state_dict(),"./data/g_{:03d}.prm".format(epoch),pickle_protocol=4)
        torch.save(d.state_dict(),"./data/d_{:03d}.prm".format(epoch),pickle_protocol=4)
        generated_img=g(fixed_z)
        save_image(generated_img,"./data/{:03d}.jpg".format(epoch))
        


files = sorted(glob.glob('./data/*.jpg'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('generating_process.gif' , save_all = True , append_images = images[1:] , duration = 200 , loop = 0)

  0%|          | 0/128 [00:00<?, ?it/s]

current epoch :  0


  0%|          | 0/128 [00:00<?, ?it/s]

runtime =  37.36971068382263 sec
expected remaining time =  3699 sec : 61 minute
current epoch :  1


  0%|          | 0/128 [00:00<?, ?it/s]

...

runtime =  35.56173825263977 sec
expected remaining time =  71 sec : 1 minute
current epoch :  98


  0%|          | 0/128 [00:00<?, ?it/s]

runtime =  35.350773334503174 sec
expected remaining time =  35 sec : 0 minute
current epoch :  99


100%|██████████| 128/128 [00:35<00:00,  3.58it/s]


runtime =  35.73525428771973 sec
expected remaining time =  0 sec : 0 minute
